## The Original EigenDecompression.EigenDecompose

See

* https://mobile.twitter.com/realize_ss/status/1615160291108745216
* https://qiita.com/lelele/items/8408410a94f5c6b8f76e

In [1]:
using LinearAlgebra
M = rand(100,100)#対角化したい行列
E, P = eigen(M)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
100-element Vector{ComplexF64}:
  -2.724962858231039 - 0.7207622109262605im
  -2.724962858231039 + 0.7207622109262605im
 -2.6548314664437656 + 0.0im
 -2.3635661717028222 + 0.0im
  -2.311359579573719 - 0.3874223153830758im
  -2.311359579573719 + 0.3874223153830758im
  -2.286803636823847 - 0.7755168568262587im
  -2.286803636823847 + 0.7755168568262587im
 -2.1332012384176133 - 1.5452028510462612im
 -2.1332012384176133 + 1.5452028510462612im
 -1.8156753210892314 - 0.735600758169247im
 -1.8156753210892314 + 0.735600758169247im
  -1.771869598468355 - 1.6181376519992137im
                     ⋮
  1.9398192405932326 - 0.5131696041292336im
  1.9398192405932326 + 0.5131696041292336im
  2.1282633990088096 - 0.891167292853587im
  2.1282633990088096 + 0.891167292853587im
   2.174694661762957 - 1.4203015873441718im
   2.174694661762957 + 1.4203015873441718im
   2.358968292344132 - 0.7001871658313624im
   2.358968292344132 

In [2]:
exp(eigen(M))

LoadError: MethodError: no method matching exp(::Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}})

[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90mspecial\[39m[90m[4mexp.jl:325[24m[39m
[0m  exp([91m::Adjoint{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:595[24m[39m
[0m  exp([91m::Transpose{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:596[24m[39m
[0m  ...


In [3]:
module EigenDecompression

export EigenDecompose, eigDecomp
using LinearAlgebra
import Base.*, Base./

#対角化された行列型
struct EigenDecompose{T<:Number} <: AbstractMatrix{T}
    P::AbstractMatrix{T}
    D::Diagonal{T}
    invP::AbstractMatrix{T}
end

#普通のMatrixを対角化する
function eigDecomp(mat::AbstractMatrix)
    E, P = eigen(mat)
    EigenDecompose(P, Diagonal(E), inv(P))
end

#EigenDecompose型に対する関数
Base.exp(eig::EigenDecompose) = EigenDecompose(eig.P, exp(eig.D), eig.invP)
*(eig::EigenDecompose, vec::AbstractVector) = eig.P * eig.D * eig.invP * vec
*(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D*sc, eig.invP)
/(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D/sc, eig.invP)

#普通のMatrixに戻す
Base.Array(eig::EigenDecompose) = eig.P * eig.D * eig.invP

end

Main.EigenDecompression

In [4]:
using .EigenDecompression

M = rand(100, 100)
eM = eigDecomp(M)
for i in 1:100
    v = rand(100)
    rnd = rand()
    @assert exp(M*rnd)*v ≈ exp(eM*rnd)*v
end

In [5]:
using BenchmarkTools

M = rand(100, 100);

#普通な方
function bench1(M)
    for i in 1:100
        v = rand(100)
        exp(M*rand())*v
    end
end

#今回実装した方
function bench2(M)
    eM = eigDecomp(M)
    for i in 1:100
        v = rand(100)
        exp(eM*rand())*v
    end
end

bench2 (generic function with 1 method)

In [6]:
@benchmark bench1(M)

BenchmarkTools.Trial: 65 samples with 1 evaluation.
 Range (min … max):  70.653 ms … 84.618 ms  ┊ GC (min … max): 4.87% … 5.74%
 Time  (median):     77.408 ms              ┊ GC (median):    2.52%
 Time  (mean ± σ):   77.587 ms ±  3.380 ms  ┊ GC (mean ± σ):  3.37% ± 1.21%

                      ▂   ▂ ▂ ▂  ▅    █               ▂        
  █▁▁▅▁▁▁▅▅█▁▅▁▅▅▅▁▅▅▅██▅████▅████▅▁▁▅█▅▅█▁▁▁▁▁█▁█▁▁▅▁█▁▁▅▁▅▅ ▁
  70.7 ms         Histogram: frequency by time        84.5 ms <

 Memory estimate: 53.80 MiB, allocs estimate: 1900.

In [7]:
@benchmark bench2(M)

BenchmarkTools.Trial: 767 samples with 1 evaluation.
 Range (min … max):  5.915 ms …   8.231 ms  ┊ GC (min … max): 0.00% … 25.51%
 Time  (median):     6.551 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.520 ms ± 287.822 μs  ┊ GC (mean ± σ):  0.43% ±  1.99%

                         ▃▅▇█                                  
  ▃▂▃▃▃▅▆▅▄▅▄▃▃▃▃▂▃▂▂▃▃▄▇████▇▇▅▄▄▄▄▃▄▄▄▃▃▂▂▂▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  5.91 ms         Histogram: frequency by time         7.4 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 1728.

## EigenDecomposedMatrices.EigenDecomposed

In [8]:
using LinearAlgebra
using BenchmarkTools

In [9]:
module EigenDecomposedMatrices

export EigenDecomposed

using LinearAlgebra

struct EigenDecomposed{
        T,
        TA<:AbstractMatrix,
        TE<:AbstractVector,
        TP<:AbstractMatrix,
        TinvP<:AbstractMatrix
    } <: AbstractMatrix{T}
    A::TA
    E::TE
    P::TP
    invP::TinvP
end

function EigenDecomposed(A::AbstractMatrix, E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
    EigenDecomposed{eltype(A), typeof(A), typeof(E), typeof(P), typeof(invP)}(A, E, P, invP)
end

function EigenDecomposed(A::AbstractMatrix)
    E, P = eigen(A)
    invP = ishermitian(A) ? P' : inv(P)
    EigenDecomposed(A, E, P, invP)
end

Base.parent(ed::EigenDecomposed) = ed.A
LinearAlgebra.eigvals(ed::EigenDecomposed) = ed.E
LinearAlgebra.eigvecs(ed::EigenDecomposed) = ed.P
Base.convert(::Type{Array}, ed::EigenDecomposed) = convert(Array, parent(ed))
for op in (:eltype, :size)
    @eval Base.$op(ed::EigenDecomposed) = $op(parent(ed))
end
Base.getindex(ed::EigenDecomposed, I...) = getindex(parent(ed), I...)

Base.:*(c::Number, ed::EigenDecomposed) = EigenDecomposed(c*ed.A, c*ed.E, ed.P, ed.invP)
Base.:*(ed::EigenDecomposed, c::Number) = EigenDecomposed(ed.A*c, ed.E*c, ed.P, ed.invP)
Base.:\(c::Number, ed::EigenDecomposed) = EigenDecomposed(c\ed.A, c\ed.E, ed.P, ed.invP)
Base.:/(ed::EigenDecomposed, c::Number) = EigenDecomposed(ed.A/c, ed.E/c, ed.P, ed.invP)
for T in (AbstractVector, AbstractMatrix)
    @eval Base.:*(ed::EigenDecomposed, v::$T) = ed.A * v
end

function exp_old(ed::EigenDecomposed)
    (; A, E, P, invP) = ed
    expE = exp.(E)
    expA = P * Diagonal(expE) * invP
    EigenDecomposed(expA, expE, P, invP)
end

LinearAlgebra.lmul!(c::Number, ed::EigenDecomposed) = (lmul!(c, parent(ed)); lmul!(c, eigvals(ed)))
LinearAlgebra.rmul!(ed::EigenDecomposed, c::Number) = (rmul!(parent(ed), c); rmul!(eigvals(ed), c))
LinearAlgebra.ldiv!(c::Number, ed::EigenDecomposed) = (ldiv!(c, parent(ed)); ldiv!(c, eigvals(ed)))
LinearAlgebra.rdiv!(ed::EigenDecomposed, c::Number) = (rdiv!(parent(ed), c); rdiv!(eigvals(ed), c))
for T in (AbstractVector, AbstractMatrix)
    @eval function LinearAlgebra.mul!(y::$T, ed::EigenDecomposed, x::$T, alpha::Number, beta::Number)
        mul!(y, parent(ed), x, alpha, beta)
    end
end

for op in (:exp, :log, :sin, :cos)
    opE = Symbol(op, "E")
    op_eigendecomposed = Symbol(op, "_eigendecomposed")
    op_eigendecomposed! = Symbol(op_eigendecomposed, "!")
    op_eigendecomposed!_doc =
        """
        $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))

        returns the `$op` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`. 
        It does not overwrite `ed` and uses `$opE` and `tmpY` as workspaces.
        """
    @eval begin
        @doc $op_eigendecomposed!_doc
        function $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))
            (; A, E, P, invP) = ed
            @. $opE = $op.(E)
            mul!(tmpY, P, Diagonal($opE))
            mul!(Y, tmpY, invP)
        end
        $op_eigendecomposed(ed::EigenDecomposed) = $op_eigendecomposed!(similar(ed.P), ed)
        Base.$op(ed::EigenDecomposed) = $op_eigendecomposed(ed)
    end
end

end

Main.EigenDecomposedMatrices

In [10]:
?rmul!

search: rmul! baremodule parentmodule promote_rule



```
rmul!(A::AbstractArray, b::Number)
```

Scale an array `A` by a scalar `b` overwriting `A` in-place.  Use [`lmul!`](@ref) to multiply scalar from left.  The scaling operation respects the semantics of the multiplication [`*`](@ref) between an element of `A` and `b`.  In particular, this also applies to multiplication involving non-finite numbers such as `NaN` and `±Inf`.

!!! compat "Julia 1.1"
    Prior to Julia 1.1, `NaN` and `±Inf` entries in `A` were treated inconsistently.


# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Matrix{Int64}:
 1  2
 3  4

julia> rmul!(A, 2)
2×2 Matrix{Int64}:
 2  4
 6  8

julia> rmul!([NaN], 0.0)
1-element Vector{Float64}:
 NaN
```

---

```
rmul!(A, B)
```

Calculate the matrix-matrix product $AB$, overwriting `A`, and return the result. Here, `B` must be of special matrix type, like, e.g., [`Diagonal`](@ref), [`UpperTriangular`](@ref) or [`LowerTriangular`](@ref), or of some orthogonal type, see [`QR`](@ref).

# Examples

```jldoctest
julia> A = [0 1; 1 0];

julia> B = UpperTriangular([1 2; 0 3]);

julia> rmul!(A, B);

julia> A
2×2 Matrix{Int64}:
 0  3
 1  2

julia> A = [1.0 2.0; 3.0 4.0];

julia> F = qr([0 1; -1 0]);

julia> rmul!(A, F.Q)
2×2 Matrix{Float64}:
 2.0  1.0
 4.0  3.0
```


In [11]:
?EigenDecomposedMatrices.exp_eigendecomposed!

exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))

returns the `exp` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.


In [12]:
?EigenDecomposedMatrices.log_eigendecomposed!

log_eigendecomposed!(Y, ed::EigenDecomposed, logE=similar(ed.E), tmpY=similar(Y))

returns the `log` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `logE` and `tmpY` as workspaces.


In [13]:
methods(EigenDecomposedMatrices.EigenDecomposed)

# 2 methods for type constructor:
 [1] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix)
     @ In[9]:24
 [2] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix, E::AbstractVector, P::AbstractMatrix, invP::AbstractMatrix)
     @ In[9]:20

In [14]:
methodswith(EigenDecomposedMatrices.EigenDecomposed)

[1] *(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:39
[2] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:40
[3] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractVector) @ Main.EigenDecomposedMatrices In[9]:44
[4] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractMatrix) @ Main.EigenDecomposedMatrices In[9]:44
[5] /(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:42
[6] \(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:41
[7] convert(::Type{Array}, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:33
[8] cos(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:84
[9] eltype(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[10] exp(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:84
[11] getindex(ed::Main.EigenDecomposedMatrices.EigenDecomposed, I...) @ Main.EigenDecomposedMatrices In[9]:37
[12] log(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:84
[13] parent(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:30
[14] sin(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:84
[15] size(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[16] eigvals(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:31
[17] eigvecs(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:32
[18] ldiv!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:56
[19] lmul!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:54
[20] mul!(y::AbstractVector, ed::Main.EigenDecomposedMatrices.EigenDecomposed, x::AbstractVector, alpha::Number, beta::Number) @ Main.EigenDecomposedMatrices In[9]:59
[21] mul!(y::AbstractMatrix, ed::Main.EigenDecomposedMatrices.EigenDecomposed, x::AbstractMatrix, alpha::Number, beta::Number) @ Main.EigenDecomposedMatrices In[9]:59
[22] rdiv!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:57
[23] rmul!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:55

In [15]:
methods(EigenDecomposedMatrices.exp_eigendecomposed!)

# 3 methods for generic function "exp_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:77
 [2] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE)
     @ In[9]:77
 [3] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE, tmpY)
     @ In[9]:77

In [16]:
methods(EigenDecomposedMatrices.log_eigendecomposed!)

# 3 methods for generic function "log_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:77
 [2] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE)
     @ In[9]:77
 [3] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE, tmpY)
     @ In[9]:77

In [17]:
A = [
    2 -1 0
    -1 2 -1
    0 -1 2
]

edA = EigenDecomposedMatrices.EigenDecomposed(A)

3×3 Main.EigenDecomposedMatrices.EigenDecomposed{Int64, Matrix{Int64}, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  2  -1   0
 -1   2  -1
  0  -1   2

In [18]:
log(edA)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [19]:
log(A)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [20]:
log(edA) ≈ log(A)

true

In [21]:
n = 2^8
M = 5I + randn(n, n)
v = randn(n)
c = randn()

edM = EigenDecomposedMatrices.EigenDecomposed(M)

Y = similar(edM.P)
expE = similar(edM.E)
tmpY = similar(Y)

y = similar(v)
tmpy = oftype(edM.E, y)
alpha = randn()
beta = randn();

In [22]:
edM

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Matrix{Float64}, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}:
  4.87255    -0.507229    0.396491   …  -0.0688048   0.0267753  -2.0903
  0.586697    5.22725     1.65916       -2.72841     0.517771   -0.650931
 -0.650661   -0.613283    6.32087       -1.62838    -0.213295    0.606555
  0.0957037  -0.984439    2.13963        1.25206     0.194758   -0.882486
 -0.491411   -2.01713     0.262164      -1.16212     0.808374   -0.209107
 -0.561059    0.169715    0.88275    …  -1.16109    -0.566131   -0.250877
  0.126085   -0.80616     0.786413      -1.66361     0.63883     0.294365
  0.353058    2.07809    -1.81548        0.883853    0.898294    0.554953
 -0.952261    1.55763    -0.258424      -0.292387    0.557137    0.630342
 -1.85629    -0.860528    0.521213      -2.23465    -0.585842    0.434833
 -0.596956   -1.23116     0.0459735  …   1.02247    -1.03702    -0.339125
  1.11443     1.14029     1.21753       -0.38720

In [23]:
dump(edM)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Matrix{Float64}, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}
  A: Array{Float64}((256, 256)) [4.872549976390289 -0.5072293514414554 … 0.026775264242302976 -2.090304406052311; 0.5866968546250569 5.227245530760838 … 0.5177706871328531 -0.6509305489017451; … ; 0.43581219010859434 -0.8097081081811991 … 4.549075588749601 -1.3103692504099345; -1.8178702348494415 0.29948283685649607 … -0.31112877791441057 4.422435324770879]
  E: Array{ComplexF64}((256,)) ComplexF64[-10.303007885224146 - 2.854389916176393im, -10.303007885224146 + 2.854389916176393im, -10.160588399135197 + 0.0im, -9.593782680771696 - 4.227501316197441im, -9.593782680771696 + 4.227501316197441im, -8.91390610422193 - 8.063557606604641im, -8.91390610422193 + 8.063557606604641im, -8.563954387474627 - 0.49857040843494554im, -8.563954387474627 + 0.49857040843494554im, -8.227730099318327 - 4.962471022646931im  …  18.44108615033121 + 3.524144477211968im, 18.614182179

In [24]:
M == parent(edM) == Matrix(edM)

true

In [25]:
M ≈ edM

true

In [26]:
c*M ≈ c*edM ≈ edM*c

true

In [27]:
c\M ≈ c\edM ≈ edM/c

true

In [28]:
(
    exp(M)
    ≈ exp(edM)
    ≈ EigenDecomposedMatrices.exp_old(edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY)
)

true

In [29]:
@show typeof(y)
mul!(y, M, v, alpha, beta) ≈ mul!(y, edM, v, alpha, beta)

typeof(y) = Vector{Float64}


true

In [30]:
@show typeof(tmpy)
(
    exp(M) * v
    ≈ exp(edM) * v
    ≈ EigenDecomposedMatrices.exp_old(edM) * v
    ≈ mul!(tmpy, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM), v)
    ≈ mul!(tmpy, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY), v)
)

typeof(tmpy) = Vector{ComplexF64}


true

In [31]:
@btime edM = EigenDecomposedMatrices.EigenDecomposed(M);

  37.752 ms (26 allocations: 3.53 MiB)


In [32]:
@btime exp($M) * $v
@btime exp($edM) * $v
@btime EigenDecomposedMatrices.exp_old($edM) * $v
@btime mul!($tmpy, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM), $v)
@btime mul!($tmpy, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM, $expE, $tmpY), $v);

  9.485 ms (16 allocations: 3.01 MiB)
  2.507 ms (7 allocations: 2.01 MiB)
  2.485 ms (7 allocations: 2.01 MiB)
  2.344 ms (3 allocations: 1.00 MiB)
  2.202 ms (0 allocations: 0 bytes)


In [33]:
n2 = 2^8
M2 = Symmetric(5I + randn(n2, n2))
v2 = randn(n)
c2 = randn()

edM2 = EigenDecomposedMatrices.EigenDecomposed(M2)

Y2 = similar(edM2.P)
expE2 = similar(edM2.E)
tmpY2 = similar(Y2)

y2 = similar(v2)
tmpy2 = oftype(edM2.E, y2)
alpha2 = randn()
beta2 = randn();

In [34]:
edM2

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Symmetric{Float64, Matrix{Float64}}, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  3.85098    0.654999   -0.22236    …   0.451596    0.924806   -0.73356
  0.654999   5.65044    -0.319709       0.895527   -1.94368    -0.357162
 -0.22236   -0.319709    4.05874       -1.38347     0.437127    0.229985
 -0.565409   0.796815    0.189045       0.0733588   2.10046     0.520188
  1.07176   -0.625197   -0.219381       0.210028    0.572497    0.412839
 -0.685181   0.0373845   1.40711    …  -1.0828      0.127981   -0.897418
 -1.4339    -0.78984     0.0412881      0.0761074   1.66859    -1.2846
  1.1693     0.624434   -0.102805       0.167902    2.15768    -0.54081
  0.419173  -1.42096    -0.764747      -0.605752   -0.99121    -1.78425
 -0.251158   0.894521   -0.800078       1.3601      0.319503    1.2047
  0.110265   1.56796     0.121427   …   1.44905     1.6954     -0.348939
  0.391195   1.34717     0.34941   

In [35]:
dump(edM2)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Symmetric{Float64, Matrix{Float64}}, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}
  A: Symmetric{Float64, Matrix{Float64}}
    data: Array{Float64}((256, 256)) [3.8509839198236686 0.6549994301800877 … 0.9248060066688447 -0.7335596326354893; 0.19812333355458134 5.650439338336893 … -1.9436821805927027 -0.3571621991841546; … ; 1.0975275361997898 -2.712274801268304 … 5.843971125545126 0.7476322646239064; 0.08737024898538669 1.490779151377127 … 2.138199034410058 5.910235294496456]
    uplo: Char 'U'
  E: Array{Float64}((256,)) [-26.22951260582353, -25.86700583128568, -25.385457524596717, -24.588820333945726, -24.436625531648033, -23.771072170581185, -23.057241407868787, -22.517899160828343, -22.236388200760558, -21.98726345846221  …  31.91892242294034, 32.51642871229246, 32.736023745647344, 33.02108560490655, 33.626011407359684, 33.73042990481509, 34.56491515347458, 35.035499594604545, 35.498385741856794, 35.97248

In [36]:
M2 == parent(edM2) == Matrix(edM2)

true

In [37]:
M2 ≈ edM2

true

In [38]:
c2*M2 ≈ c2*edM2 ≈ edM2*c2

true

In [39]:
c2\M2 ≈ c2\edM2 ≈ edM2/c2

true

In [40]:
(
    exp(M2)
    ≈ exp(edM2)
    ≈ EigenDecomposedMatrices.exp_old(edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2)
)

true

In [41]:
@show typeof(y2)
mul!(y2, M2, v2, alpha2, beta2) ≈ mul!(y2, edM2, v2, alpha2, beta2)

typeof(y2) = Vector{Float64}


true

In [42]:
@show typeof(tmpy2)
(
    exp(M2) * v2
    ≈ exp(edM2) * v2
    ≈ EigenDecomposedMatrices.exp_old(edM2) * v2
    ≈ mul!(tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2), v2)
    ≈ mul!(tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2), v2)
)

typeof(tmpy2) = Vector{Float64}


true

In [43]:
@btime edM2 = EigenDecomposedMatrices.EigenDecomposed(M2);

  6.068 ms (14 allocations: 1.59 MiB)


In [44]:
@btime exp($M2) * $v2
@btime exp($edM2) * $v2
@btime EigenDecomposedMatrices.exp_old($edM2) * $v2
@btime mul!($tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2), $v2)
@btime mul!($tmpy2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2, $expE2, $tmpY2), $v2);

  6.704 ms (19 allocations: 2.60 MiB)
  669.000 μs (6 allocations: 1.00 MiB)
  668.100 μs (6 allocations: 1.00 MiB)
  647.000 μs (3 allocations: 514.17 KiB)
  618.200 μs (0 allocations: 0 bytes)
